In [1]:
import sys, time
import numpy as np
import arkouda as ak

In [2]:
ak.connect(server="localhost", port=5555)

4.3.1
psp =  tcp://localhost:5555
connected to tcp://localhost:5555


In [3]:
# files and locations on my(mike's) laptop
hdffiles = ["/Users/valton/lanl_netflow/hdf5/netflow_day-02-00.hdf",
           "/Users/valton/lanl_netflow/hdf5/netflow_day-02-01.hdf",
           "/Users/valton/lanl_netflow/hdf5/netflow_day-02-02.hdf",
           "/Users/valton/lanl_netflow/hdf5/netflow_day-02-03.hdf",
           "/Users/valton/lanl_netflow/hdf5/netflow_day-02-04.hdf",
           "/Users/valton/lanl_netflow/hdf5/netflow_day-02-05.hdf"]

In [4]:
fields = ['srcIP', 'dstIP', 'srcPort', 'dstPort', 'start']

In [5]:
%time nfDF = {field: ak.read_hdf(field, hdffiles) for field in fields}

CPU times: user 1.57 ms, sys: 1.65 ms, total: 3.21 ms
Wall time: 6.11 s


In [6]:
print(nfDF['start'])
print(nfDF['srcIP'])
print(nfDF['dstIP'])
print(nfDF['srcPort'])
print(nfDF['dstPort'])

[118781 118783 118785 ... 140369 140369 140369]
[34111590301 34111676798 48764590996 ... 34112191431 34112191431 34112191431]
[34111773101 34112055194 34111367844 ... 34111545319 31712083968 31712083968]
[5507 3137 5060 ... 78620 44270 49614]
[46272 445 5060 ... 95765 80 80]


In [7]:
print(nfDF)

{'srcIP': array([34111590301, 34111676798, 48764590996, ..., 34112191431, 34112191431, 34112191431]), 'dstIP': array([34111773101, 34112055194, 34111367844, ..., 34111545319, 31712083968, 31712083968]), 'srcPort': array([5507, 3137, 5060, ..., 78620, 44270, 49614]), 'dstPort': array([46272, 445, 5060, ..., 95765, 80, 80]), 'start': array([118781, 118783, 118785, ..., 140369, 140369, 140369])}


In [8]:
print(ak.info(ak.AllSymbols))

name:"id_5" dtype:"int64" size:60000000 ndim:1 shape:(60000000) itemsize:8
name:"id_4" dtype:"int64" size:60000000 ndim:1 shape:(60000000) itemsize:8
name:"id_3" dtype:"int64" size:60000000 ndim:1 shape:(60000000) itemsize:8
name:"id_2" dtype:"int64" size:60000000 ndim:1 shape:(60000000) itemsize:8
name:"id_1" dtype:"int64" size:60000000 ndim:1 shape:(60000000) itemsize:8



In [9]:
%time u,c = ak.unique(nfDF['srcIP'], return_counts=True)

CPU times: user 981 µs, sys: 1.35 ms, total: 2.33 ms
Wall time: 13.8 s


In [10]:
print(u.size,u)
print(c.size,c)

25559 [34112110600 48764978487 34111400624 ... 48764444540 34111644622 34111308561]
25559 [3976 176 3960 ... 183 561 2159]


In [11]:
%time u,c = ak.unique(nfDF['dstIP'], return_counts=True)

CPU times: user 1.41 ms, sys: 2.02 ms, total: 3.44 ms
Wall time: 14.3 s


In [12]:
print(u.size,u)
print(c.size,c)

23790 [34111337455 34111947526 48764616254 ... 34112119691 48764677226 48764683812]
23790 [39 72 7 ... 56 2 4]


In [13]:
%time u,c = ak.unique(nfDF['srcPort'], return_counts=True)

CPU times: user 986 µs, sys: 1.71 ms, total: 2.69 ms
Wall time: 1.39 s


In [14]:
print(u.size,u)
print(c.size,c)

64899 [2 4 5 ... 99996 99997 99998]
64899 [188 223 2518 ... 284 220 220]


In [15]:
%time u,c = ak.unique(nfDF['dstPort'], return_counts=True)

CPU times: user 1.05 ms, sys: 2.18 ms, total: 3.23 ms
Wall time: 1.24 s


In [16]:
print(u.size,u)
print(c.size,c)

59664 [2 4 7 ... 99996 99997 99998]
59664 [4 11 4698 ... 7 5 5]


In [17]:
print((nfDF['srcPort'] == 8080).sum())
print((nfDF['dstPort'] == 8080).sum())

2508
418986


In [18]:
s = u # pick off some dest ports
print(s)
%time t = ak.in1d(nfDF['dstPort'], s) # bool vector of where they are found in the data
print(t.sum())

[2 4 7 ... 99996 99997 99998]
CPU times: user 1.41 ms, sys: 1.12 ms, total: 2.53 ms
Wall time: 10 s
60000000


In [20]:
s = u.to_ndarray() # pick off some dest ports
dp = nfDF['dstPort'].to_ndarray()
print(s)
%time t = np.in1d(dp, s) # bool vector of where they are found in the data
print(t.sum())

[    2     4     7 ... 99996 99997 99998]
CPU times: user 7.46 s, sys: 1.02 s, total: 8.48 s
Wall time: 8.49 s
60000000


In [21]:
a = nfDF['start']
aMin = a.min()
aMax = a.max()
print(aMin,aMax, aMax - aMin)
%time h = ak.histogram(a,bins=100)
print(h)
print(h.sum(), a.size)

118781 140369 21588
CPU times: user 700 µs, sys: 268 µs, total: 968 µs
Wall time: 126 ms
[87 260 221 ... 507133 811672 769215]
60000000 60000000


In [66]:
#ak.shutdown()
#ak.disconnect()